In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.externals import joblib

In [2]:
test_data=pd.read_excel('test.xlsx')
sample_data=pd.read_csv('sample.csv')

In [3]:
def create_test_data(df):
    for col in df.columns:
        ide=[]
        for i in range(df.shape[0]):
            if df[col][i]=='?':
                ide.append(i)
        for i in ide:
            df[col][i]=np.nan
            
    #drop 'normalized-losses' because I have droped it from the training data
    
    df=df.drop('normalized-losses',axis=1)
    
    # Use bellow process if there is any null value in the test data
    
    """
    #handling missing values
    
    df['stroke']=df['stroke'].fillna(df['stroke'].median())
    
    df['bore']=df['bore'].fillna(df['bore'].median())
    
    df['peak-rpm']=df['peak-rpm'].fillna(df['peak-rpm'].median())
    
    df['num-of-doors']=df['num-of-doors'].fillna(df['num-of-doors'].value_counts().index[0])
    """
    
    # label encoder for converting categorical data into numerical data
    
    le=LabelEncoder()
    for col in df.columns:
        if df[col].dtype=='object':
            le.fit(df[col].astype(str))
            df[col]=le.transform(df[col].astype(str))
            
            
    
    
    return df

In [4]:
test=create_test_data(test_data)

/home/vs/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
test.shape

(21, 25)

# Load trained model

In [6]:
filename='finalized_model.sav'
loaded_model=joblib.load(filename)


In [7]:
y_predict=loaded_model.predict(test)

In [8]:
sample_data.head(10)

,id,price,price-category
0,0,5572,Low
1,1,18420,High
2,2,7395,Low
3,3,8358,Medium
4,4,19045,High
5,5,8013,Medium


In [9]:
y_pred=[int(val) for val in y_predict]

In [10]:
ouput=pd.DataFrame()
ouput['id']=test['id']
ouput['price']=y_pred

In [11]:
ouput.head(10)

,id,price
0,0,8718
1,1,20890
2,2,11280
3,3,11117
4,4,23248
5,5,12932
6,6,9430
7,21,9673
8,22,27354
9,23,13502


In [12]:
max_price=np.max(y_pred)
max_price

37223

In [13]:
min_price=np.min(y_pred)
min_price

8718

In [14]:
diff=max_price - min_price
d=int(diff/3)

In [15]:
p1=min_price
p2=p1+d
p3=p2+d
p4=max_price

In [16]:
price_category=[]
for val in y_pred:
    if val>=p1 and val<p2:
        price_category.append('Low')
    elif val>=p2 and val<p3:
        price_category.append('Medium')
    else:
        price_category.append('High')
        

In [17]:
ouput['price-category']=price_category

In [18]:
ouput.head()

,id,price,price-category
0,0,8718,Low
1,1,20890,Medium
2,2,11280,Low
3,3,11117,Low
4,4,23248,Medium


In [19]:
ouput.to_csv('ouput.csv',index=False)